In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U spacy
!pip install spacy_transformers

  Using cached spacy-3.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1


In [3]:
!pip install spacy==3.5.1

  Using cached spacy-3.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.1.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (919 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.3
    Uninstalling thinc-8.2.3:
      Successfully uninstalled thinc-8.2.3
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.4
    Uninstalling spacy-3.7.4:
      Successfully uninstalled spacy-3.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.5.1 which is incompatible.


In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [5]:
spacy.__version__


'3.5.1'

In [15]:
!nvidia-smi

Thu May  2 11:44:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
cv_data=json.load(open('/content/resumeparser2151/dataset/dataset.json', 'r'))

In [17]:
len(cv_data)


1014

In [18]:
cv_data[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [19]:
!python -m spacy init fill-config /content/resumeparser2151/config/base_config.cfg /content/resumeparser2151/config/config.cfg

2024-05-02 11:49:04.337539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 11:49:04.337613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 11:49:04.338925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 11:49:05.418116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_no

In [27]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = set()  # Using a set to track indices for better performance

        for start, end, label in annot:
            if not any(idx in entity_indices for idx in range(start, end)):  # Check for overlap
                entity_indices.update(range(start, end))
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
                if span is None:
                    file.write(f"Error: Unable to form span for {start}-{end} in text: {text[:50]}...\n")
                else:
                    ents.append(span)

        doc.ents = ents  # Set the document entities
        db.add(doc)  # Add the processed document to DocBin

    return db

In [30]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.3)

In [28]:
len(train),len(test)

(709, 305)

In [33]:
file = open('/content/resumeparser2151/model/train_file.txt', 'w')

# Process training data
train_db = get_spacy_doc(file, train)
train_db.to_disk('/content/resumeparser2151/model/train_data.spacy')

# Process testing data
test_db = get_spacy_doc(file, test)
test_db.to_disk('/content/resumeparser2151/model/test_data.spacy')

file.close()

100%|██████████| 305/305 [00:03<00:00, 93.30it/s]


In [34]:
!python -m spacy train /content/resumeparser2151/config/config.cfg --output /content/resumeparser2151/model/output --paths.train /content/resumeparser2151/model/train_data.spacy --paths.dev /content/resumeparser2151/model/test_data.spacy --gpu-id 0

2024-05-02 12:10:28.683395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 12:10:28.683449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 12:10:28.684831: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 12:10:29.800270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/resumeparser2151/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: Use

In [35]:
nlp=spacy.load('/content/resumeparser2151/model/output/model-best')

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [36]:
!pip install PyMuPDF

In [37]:
import sys,fitz

In [38]:
fname=r'/content/resumeparser2151/test/AkashDesai.pdf'
doc=fitz.open(fname)

In [39]:
text= "  "
for page in doc:
    text= text + str(page.get_text())

In [40]:
text

"  AKASH DESAI \n \n \nLinkedIn \n \n• https://www.linkedin.com/in/akash-desai-587432228/  \n \nProfessional Summary \n \nExperienced Python Developer with a strong track record of designing, developing, and maintaining software solutions. \nProficient in Python programming, I excel in building web applications, integrating APIs, and optimizing database \nperformance. My dedication to delivering clean, efficient code and my ability to collaborate effectively within cross-functional \nteams have contributed to the successful creation of scalable and maintainable software solutions. Eager to apply my skills \nand passion for Python development to drive the success of future projects. \nSkills \n \n \n\uf0b7 Python Development \n\uf0b7 Product Development \n\uf0b7 Web Development \n\uf0b7 Database Management \n\uf0b7 Frontend Technologies \n\uf0b7 Version Control \n \n\uf0b7 Testing and Debugging \n\uf0b7 Software Architecture \n\uf0b7 Security \n\uf0b7 Deployment and DevOps \n\uf0b7 Agil

In [2]:
doc = nlp(text)
for ent in doc.ents:
    print(ent,"->>>>>>>>>>",ent.label_)

NameError: name 'nlp' is not defined